### **Fig 7: observations and SMRT simulations per pit**

In [ ]:
import pandas as pd
import xarray as xr
import numpy as np
%matplotlib widget
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.ticker import AutoMinorLocator

In [ ]:
%load_ext autoreload
%aimport aesop
%aimport run_smrt_with_realistic_atmosphere
%autoreload 1

Load SMRT results:

In [ ]:
smrt_c087 = xr.open_dataset('../Other/smrt_results_atmos/smrt_atmos_tb_C087.nc')
smrt_c090 = xr.open_dataset('../Other/smrt_results_atmos/smrt_atmos_tb_C090.nc')

Get 'pit order' to ensure all datasets are plotted for same pits:

In [ ]:
pit_order = smrt_c090.snowpack.values
# Group MetS with A04
pit_order = np.insert(pit_order, 12, 'MetS')
pit_order = pit_order[:-1]

Load observations:

In [ ]:
obs_c087 = aesop.get_obs_per_aoi('C087')
obs_c090 = aesop.get_obs_per_aoi('C090')

Load pits with topography and re-order index to match smrt results:

In [ ]:
pits = aesop.get_pit_topography()
pits = pits.reindex(pit_order)

Load ground based observations:

In [ ]:
sbr = pd.read_csv('../Data/ground_obs/more89forMel.csv')
tb89v0 = run_smrt_with_realistic_atmosphere.ground_based_data_corrected(sbr, '89V0', pits)

### Plot ground and airborne observations and smrt results:

In [ ]:
def plot_results(smrt, obs, channel, flight, ground=False):
    fig = plt.figure(figsize=(10,6))

    # SMRT
    smrt_tb = smrt.sel(channel=channel).reindex({'snowpack': pit_order})

    spl = plt.errorbar(pit_order, smrt_tb['base'],
                       yerr=[smrt_tb['base']-smrt_tb['low'], smrt_tb['high']-smrt_tb['base']],
                       fmt='H', color='b', label='SMRT Sims', capsize=0, markeredgecolor='k',
                       ecolor='dodgerblue', elinewidth=3)

    # OBS - plot observations for relevant topography type within each AOI
    tb_obs = []
    for p in pit_order:
        topo = pits.loc[p].topo_type
        p = 'A04' if p == 'MetS' else p

        tb_obs.append(obs.loc[p[:3]][channel][obs.loc[p[:3]][channel].topo_type==topo].TB)

    boxprops = dict(color='k', linewidth=1)
    medianprops = dict(color='k', linewidth=1)
    flierprops = dict(marker='.', markeredgecolor='k',linestyle='none')
    positions = range(0, len(tb_obs))

    apl = plt.boxplot(tb_obs, positions=positions, medianprops=medianprops,
                      boxprops=boxprops, flierprops=flierprops)
    
    # if requested plot ground observations (89 GHz only)
    if ground == True:
        gpl=plt.errorbar(positions, tb89v0[:,0], yerr = [tb89v0[:,1],tb89v0[:,2]], fmt='o',
             color='salmon', label='Ground Data', capsize=3, markeredgecolor='k')
    
    # plot legend
    box_patch = mpatches.Patch(edgecolor='k', facecolor='w', label='Airborne Data ('+flight+')')
    if ground == True:
        plt.legend(handles=[(box_patch), gpl, spl], loc='lower center', fontsize=14)
    else:
        plt.legend(handles=[(box_patch), spl], loc='lower center', fontsize=14)
        
    locs, labels = plt.xticks()
    plt.xticks(locs, pit_order, rotation=90, fontsize=12)
    plt.xlabel('Pit', fontsize=14)
    
    plt.yticks(fontsize=12)
    if channel in ['M16-89', 'M17-157']:
        plt.ylabel('TB'+channel[4:] +' @0 deg [K]', fontsize=14)
    elif channel == '118+5.0':
        plt.ylabel('TB118$\pm$5.0 @0 deg [K]', fontsize=14)
    elif channel == 'M20-183+7':
        plt.ylabel('TB183$\pm$7 @0 deg [K]', fontsize=14)
    elif channel == '243':
        plt.ylabel('TB243 @0 deg [K]', fontsize=14)
        
    plt.tight_layout(pad=2)
    plt.show()

In [ ]:
plot_results(smrt_c087, obs_c087, 'M16-89', 'C087', True)

In [ ]:
plot_results(smrt_c090, obs_c090, 'M16-89', 'C090', True)

In [ ]:
plot_results(smrt_c087, obs_c087, 'M20-183+7', 'C087')

In [ ]:
plot_results(smrt_c090, obs_c090, 'M20-183+7', 'C090')

### Plot multiple channels:

In [ ]:
channels = ['M16-89','118+5.0','M17-157','M20-183+7','243']

In [ ]:
def plot_all_channels(smrt, obs, flight):
    fig, axes = plt.subplots(5,1,figsize=(10,20))

    for channel, ax in zip(channels, axes):

        # SMRT
        smrt_tb = smrt.sel(channel=channel).reindex({'snowpack': pit_order})

        spl = ax.errorbar(pit_order, smrt_tb['base'],
                           yerr=[smrt_tb['base']-smrt_tb['low'], smrt_tb['high']-smrt_tb['base']],
                           fmt='H', color='b', label='SMRT Sims', capsize=0, markeredgecolor='k',
                           ecolor='dodgerblue', elinewidth=3)

        # OBS
        tb_obs = []
        for p in pit_order:
            topo = pits.loc[p].topo_type
            p = 'A04' if p == 'MetS' else p

            tb_obs.append(obs.loc[p[:3]][channel][obs.loc[p[:3]][channel].topo_type==topo].TB)

        boxprops = dict(color='k', linewidth=1)
        medianprops = dict(color='k', linewidth=1)
        flierprops = dict(marker='.', markeredgecolor='k',linestyle='none')
        positions = range(0, len(tb_obs))

        apl = ax.boxplot(tb_obs, positions=positions, medianprops=medianprops,
                          boxprops=boxprops, flierprops=flierprops)


        ax.set_xticklabels(pit_order, rotation=90)
        ax.set_xlabel('Pit #')

        box_patch = mpatches.Patch(edgecolor='k', facecolor='w', label='Airborne Data ('+flight+')')
        ax.legend(handles=[(box_patch), spl], loc='lower center')
        
    axes[0].set_ylabel('TB89 @0 deg [K]')
    axes[1].set_ylabel('TB118$\pm$5.0 @0 deg [K]')
    axes[2].set_ylabel('TB157 @0 deg [K]')
    axes[3].set_ylabel('TB183$\pm$7 @0 deg [K]')
    axes[4].set_ylabel('TB243 @0 deg [K]')

    fig.subplots_adjust(hspace=0)
    plt.show()

In [ ]:
plot_all_channels(smrt_c087, obs_c087, 'C087')

In [ ]:
plot_all_channels(smrt_c090, obs_c090, 'C090')

### Plot C087 and C090 together:

In [ ]:
fig, ax = plt.subplots(1,figsize=(16,6))
channel = 'M16-89'

tb_obs = []
for p in pit_order:
    topo = pits.loc[p].topo_type
    p = 'A04' if p == 'MetS' else p

    tb_obs.append(obs_c087.loc[p[:3]][channel][obs_c087.loc[p[:3]][channel].topo_type==topo].TB)
    tb_obs.append(obs_c090.loc[p[:3]][channel][obs_c090.loc[p[:3]][channel].topo_type==topo].TB)
    
apl = ax.boxplot(tb_obs, positions=range(0, len(tb_obs)), showfliers=False)
    
smrt_tb_87 = smrt_c087.sel(channel=channel).reindex({'snowpack': pit_order})
smrt_tb_90 = smrt_c090.sel(channel=channel).reindex({'snowpack': pit_order})

spl = ax.errorbar(np.arange(0, len(pit_order)*2, 2), smrt_tb_87['base'],
                       yerr=[smrt_tb_87['base']-smrt_tb_87['low'], smrt_tb_87['high']-smrt_tb_87['base']],
                       fmt='H', color='b', label='SMRT Sims', capsize=0, markeredgecolor='k',
                       ecolor='dodgerblue', elinewidth=3)  
spl = ax.errorbar(np.arange(1, len(pit_order)*2, 2), smrt_tb_90['base'],
                       yerr=[smrt_tb_90['base']-smrt_tb_90['low'], smrt_tb_90['high']-smrt_tb_90['base']],
                       fmt='H', color='b', label='SMRT Sims', capsize=0, markeredgecolor='k',
                       ecolor='dodgerblue', elinewidth=3)      
    

ax.xaxis.set_ticks(np.arange(0.5, len(pit_order)*2, 2))
ax.set_xticklabels(pit_order, rotation=90)

ax.xaxis.set_minor_locator(AutoMinorLocator(2))
ax.grid(b=True, axis='x', which='minor', linestyle='--', alpha=0.5)

ax.set_xticklabels(pit_order, rotation=90)
ax.set_xlabel('Pit #')
ax.set_ylabel('TB89 @0 deg [K]')

gpl=ax.errorbar(np.arange(0, len(pit_order)*2, 2), tb89v0[:,0], yerr = [tb89v0[:,1],tb89v0[:,2]], fmt='o',
         color='tomato', label='Ground Data', capsize=3, markeredgecolor='k')
gpl2=ax.errorbar(np.arange(1, len(pit_order)*2, 2), tb89v0[:,0], yerr = [tb89v0[:,1],tb89v0[:,2]], fmt='o',
         color='tomato', label='Ground Data', capsize=3, markeredgecolor='k')

box_patch = mpatches.Patch(edgecolor='k', facecolor='w', label='Airborne Data')
ax.legend(handles=[(box_patch), spl, gpl], loc='lower center')


fig.subplots_adjust(hspace=0)
plt.show()

In [ ]:
fig, ax = plt.subplots(1,figsize=(16,6))

channel = 'M20-183+7'

tb_obs = []
for p in pit_order:
    topo = pits.loc[p].topo_type
    p = 'A04' if p == 'MetS' else p

    tb_obs.append(obs_c087.loc[p[:3]][channel][obs_c087.loc[p[:3]][channel].topo_type==topo].TB)
    tb_obs.append(obs_c090.loc[p[:3]][channel][obs_c090.loc[p[:3]][channel].topo_type==topo].TB)
    
apl = ax.boxplot(tb_obs, positions=range(0, len(tb_obs)), showfliers=False)
    
smrt_tb_87 = smrt_c087.sel(channel=channel).reindex({'snowpack': pit_order})
smrt_tb_90 = smrt_c090.sel(channel=channel).reindex({'snowpack': pit_order})

spl = ax.errorbar(np.arange(0, len(pit_order)*2, 2), smrt_tb_87['base'],
                       yerr=[smrt_tb_87['base']-smrt_tb_87['low'], smrt_tb_87['high']-smrt_tb_87['base']],
                       fmt='H', color='b', label='SMRT Sims', capsize=0, markeredgecolor='k',
                       ecolor='dodgerblue', elinewidth=3)  
spl = ax.errorbar(np.arange(1, len(pit_order)*2, 2), smrt_tb_90['base'],
                       yerr=[smrt_tb_90['base']-smrt_tb_90['low'], smrt_tb_90['high']-smrt_tb_90['base']],
                       fmt='H', color='b', label='SMRT Sims', capsize=0, markeredgecolor='k',
                       ecolor='dodgerblue', elinewidth=3)      
    

ax.xaxis.set_ticks(np.arange(0.5, len(pit_order)*2, 2))
ax.set_xticklabels(pit_order, rotation=90)

ax.xaxis.set_minor_locator(AutoMinorLocator(2))
ax.grid(b=True, axis='x', which='minor', linestyle='--', alpha=0.5)

ax.set_xticklabels(pit_order, rotation=90)
ax.set_xlabel('Pit #')
ax.set_ylabel('TB183$\pm$7 @0 deg [K]')

box_patch = mpatches.Patch(edgecolor='k', facecolor='w', label='Airborne Data')
ax.legend(handles=[(box_patch), spl], loc='lower center')


fig.subplots_adjust(hspace=0)
plt.show()

In [ ]:
fig, axes = plt.subplots(5,1,figsize=(16,20))

for channel, ax in zip(channels, axes):
    tb_obs = []
    for p in pit_order:
        topo = pits.loc[p].topo_type
        p = 'A04' if p == 'MetS' else p

        tb_obs.append(obs_c087.loc[p[:3]][channel][obs_c087.loc[p[:3]][channel].topo_type==topo].TB)
        tb_obs.append(obs_c090.loc[p[:3]][channel][obs_c090.loc[p[:3]][channel].topo_type==topo].TB)

    apl = ax.boxplot(tb_obs, positions=range(0, len(tb_obs)), showfliers=False)

    smrt_tb_87 = smrt_c087.sel(channel=channel).reindex({'snowpack': pit_order})
    smrt_tb_90 = smrt_c090.sel(channel=channel).reindex({'snowpack': pit_order})

    spl = ax.errorbar(np.arange(0, len(pit_order)*2, 2), smrt_tb_87['base'],
                           yerr=[smrt_tb_87['base']-smrt_tb_87['low'], smrt_tb_87['high']-smrt_tb_87['base']],
                           fmt='H', color='b', label='SMRT Sims', capsize=0, markeredgecolor='k',
                           ecolor='dodgerblue', elinewidth=3)  
    spl = ax.errorbar(np.arange(1, len(pit_order)*2, 2), smrt_tb_90['base'],
                           yerr=[smrt_tb_90['base']-smrt_tb_90['low'], smrt_tb_90['high']-smrt_tb_90['base']],
                           fmt='H', color='b', label='SMRT Sims', capsize=0, markeredgecolor='k',
                           ecolor='dodgerblue', elinewidth=3)      


    ax.xaxis.set_ticks(np.arange(0.5, len(pit_order)*2, 2))
    ax.set_xticklabels(pit_order, rotation=90)

    ax.xaxis.set_minor_locator(AutoMinorLocator(2))
    ax.grid(b=True, axis='x', which='minor', linestyle='--', alpha=0.5)

    ax.set_xticklabels(pit_order, rotation=90)
    ax.set_xlabel('Pit #')

    box_patch = mpatches.Patch(edgecolor='k', facecolor='w', label='Airborne Data')
    ax.legend(handles=[(box_patch), spl], loc='lower center')
    if channel in ['M16-89', 'M17-157']:
        ax.set_ylabel('TB'+channel[4:] +' @0 deg [K]')
    elif channel == '118+5.0':
        ax.set_ylabel('TB118$\pm$5.0 @0 deg [K]')
    elif channel == 'M20-183+7':
        ax.set_ylabel('TB183$\pm$7 @0 deg [K]')
    elif channel == '243':
        ax.set_ylabel('TB243 @0 deg [K]')
        
fig.subplots_adjust(hspace=0)
plt.show()

In [ ]:
fig, axes = plt.subplots(2,1,figsize=(16,8))

for channel, ax in zip(['118+5.0', 'M20-183+7'], axes):
    tb_obs = []
    for p in pit_order:
        topo = pits.loc[p].topo_type
        p = 'A04' if p == 'MetS' else p

        tb_obs.append(obs_c087.loc[p[:3]][channel][obs_c087.loc[p[:3]][channel].topo_type==topo].TB)
        tb_obs.append(obs_c090.loc[p[:3]][channel][obs_c090.loc[p[:3]][channel].topo_type==topo].TB)

    apl = ax.boxplot(tb_obs, positions=range(0, len(tb_obs)), showfliers=False)

    smrt_tb_87 = smrt_c087.sel(channel=channel).reindex({'snowpack': pit_order})
    smrt_tb_90 = smrt_c090.sel(channel=channel).reindex({'snowpack': pit_order})

    spl = ax.errorbar(np.arange(0, len(pit_order)*2, 2), smrt_tb_87['base'],
                           yerr=[smrt_tb_87['base']-smrt_tb_87['low'], smrt_tb_87['high']-smrt_tb_87['base']],
                           fmt='H', color='b', label='SMRT Sims', capsize=0, markeredgecolor='k',
                           ecolor='dodgerblue', elinewidth=3)  
    spl = ax.errorbar(np.arange(1, len(pit_order)*2, 2), smrt_tb_90['base'],
                           yerr=[smrt_tb_90['base']-smrt_tb_90['low'], smrt_tb_90['high']-smrt_tb_90['base']],
                           fmt='H', color='b', label='SMRT Sims', capsize=0, markeredgecolor='k',
                           ecolor='dodgerblue', elinewidth=3)      


    ax.xaxis.set_ticks(np.arange(0.5, len(pit_order)*2, 2))
    ax.set_xticklabels(pit_order, rotation=90)

    ax.xaxis.set_minor_locator(AutoMinorLocator(2))
    ax.grid(b=True, axis='x', which='minor', linestyle='--', alpha=0.5)

    ax.set_xticklabels(pit_order, rotation=90)
    ax.set_xlabel('Pit #')

    box_patch = mpatches.Patch(edgecolor='k', facecolor='w', label='Airborne Data')
    ax.legend(handles=[(box_patch), spl], loc='lower center')
    if channel in ['M16-89', 'M17-157']:
        ax.set_ylabel('TB'+channel[4:] +' @0 deg [K]')
    elif channel == '118+5.0':
        ax.set_ylabel('TB118$\pm$5.0 @0 deg [K]')
    elif channel == 'M20-183+7':
        ax.set_ylabel('TB183$\pm$7 @0 deg [K]')
    elif channel == '243':
        ax.set_ylabel('TB243 @0 deg [K]')
        
fig.subplots_adjust(hspace=0)
plt.show()